<a href="https://colab.research.google.com/github/neroblack4life/2019AzureMigrateYourApps/blob/master/Copy_of_Untitled3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# llama.cpp Setup

# 1. Install dependencies
!apt-get update
!apt-get install -y make g++ git cmake

# 2. Clone llama.cpp repo
!git clone https://github.com/ggerganov/llama.cpp
%cd llama.cpp

# 3. Build
!make

# 4. Download model weights into models folder
# Instructions:
# - Obtain original LLaMA model weights and place in ./models folder
# - For BPE models, get vocab.json and add to models folder

# 5. Convert model
!python3 convert.py models/7B/

# 6. Quantize model
!./quantize ./models/7B/ggml-model-f16.bin ./models/7B/ggml-model-q4_0.bin q4_0

# 7. Run inference
print("Loading model...")
!./main -m ./models/7B/ggml-model-q4_0.bin -n 128 -p "Hello world"

In [ ]:
# pyllama Setup

# 1. Install pyllama
!pip install pyllama

# 2. Download model
!python -m pyllama.download --model_size 7B

# 3. Quantize model (optional)
!python -m pyllama.llama_quant --model_size 7B --save pyllama-7B4b.pt

# 4. Run inference
import pyllama

llama = pyllama.LLama(model_name="7B", quantize="pyllama-7B4b.pt")

response = llama("Hello world", use_gpu=False)
print(response)

In [ ]:
# Install llama-api-server
!pip install llama-api-server
!pip install llama-api-server[pyllama]

!cat > config.yml << EOF
models:
  completions:
    text-ada-002:
      type: llama_cpp
      params:
        path: /absolute/path/to/your/7B/ggml-model-q4_0.bin
    text-davinci-002:
      type: pyllama_quant
      params:
        path: /absolute/path/to/your/pyllama-7B4b.pt
    text-davinci-003:
      type: pyllama
      params:
        ckpt_dir: /absolute/path/to/your/7B/
        tokenizer_path: /absolute/path/to/your/tokenizer.model
      # keep to 1 instance to speed up loading of model
  embeddings:
    text-embedding-davinci-002:
      type: pyllama_quant
      params:
        path: /absolute/path/to/your/pyllama-7B4b.pt
      min_instance: 1
      max_instance: 1
      idle_timeout: 3600
    text-embedding-ada-002:
      type: llama_cpp
      params:
        path: /absolute/path/to/your/7B/ggml-model-q4_0.bin
EOF

!echo "SOME_TOKEN" > tokens.txt

In [ ]:
# Call with openai-python

import openai

# Set API key and base URL
openai.api_key = "wWw2gocigV4a6wuUq4wNOy4wHPHz5VMw"
openai.api_base = "http://127.0.0.1:5000/v1"

# Start the server
!python -m llama_api_server &

# Test completions
response = openai.Completion.create(
  engine="text-ada-002",
  prompt="Hello",
  max_tokens=5
)

print(response)

# Test embeddings
import requests
import json

url = "http://127.0.0.1:5000/v1/embeddings"
headers = {"Content-Type": "application/json", "Authorization": f"Bearer {openai.api_key}"}
data = json.dumps({"model": "text-embedding-ada-002", "input": "Hello world"})

response = requests.post(url, headers=headers, data=data)
print(response.json())